In [25]:
import os

files = [(file, os.path.join(root, file)) for root, subdirs, files in os.walk('files') for file in files]

In [26]:
len(files)

13088

In [27]:
import re

valid_file_regex = re.compile(r'^[a-zA-Z0-9_.-]*$')

invalid_files = list(filter(lambda x: re.match(valid_file_regex, x[0]) == None, files))


In [28]:
len(invalid_files)

3468

In [29]:
# loading csv files in memory
import pandas as pd

csvs = {}

CSV_FILE_PATH = 'csv'

for f in os.listdir(CSV_FILE_PATH):
    csv = pd.read_csv(os.path.join(CSV_FILE_PATH, f), dtype=str)
    csvs[f] = csv


In [30]:
from urllib.parse import quote

# Building a table to store invalid file names and it's reference in csv

# The key of the dict is "File path" 
# The value is a tuple of the form: (Encoded file path, filename, List of references)
# List of references is list containing tuple of the form: (csv file name, column number, row number)

invalid_file_table = {}

# iterating over each invalid file
for fileName, filePath in invalid_files:
    encodedPath = quote(filePath)
    listReferences = []
    for csv_file_name in csvs:
        csv_dataframe = csvs[csv_file_name]
        num_cols = len(csv_dataframe.columns)

        for col_i in range(num_cols):
            col_series = csv_dataframe.iloc[:,col_i].dropna()
            contains_series = col_series.str.contains(encodedPath)

            contains_indices = list(contains_series[contains_series].index)

            for row_j in contains_indices:
                listReferences.append((csv_file_name, col_i, row_j))

    invalid_file_table[filePath] = (encodedPath, fileName, listReferences)




In [19]:
pd.set_option('display.max_colwidth', None)
type(csvs['migration-article-content-download-file.csv'].iloc[1123, 2])

str

In [31]:

existing_files_set = set(map(lambda x: x[1], files))
new_files_set = set()

rename_table = {}

valid_char = re.compile(r'[a-zA-Z0-9_.-]')

path_gen = lambda op, nn: "/".join( op.split('/')[:-1] + [ nn ])

for path in invalid_file_table:

    name = invalid_file_table[path][1]
    new_name = "".join(map( lambda x : '_'  if valid_char.match(x) == None else x, name ))

    new_path = path_gen(path, new_name)

    counter = 1
    append_counter = False

    while new_path in existing_files_set or new_path in new_files_set:
        name_splits = new_name.split('.')
        idx = -2 if len(name_splits) > 1 else -1
        frag = name_splits[idx][:-4] if append_counter else name_splits[idx]
        frag += f'_{counter:03d}'
        name_splits[idx] = frag
        new_name = ".".join(name_splits)
        new_path = path_gen(new_path, new_name)
        append_counter = True
        counter += 1

    new_files_set.add(new_path)    

    rename_table[path] = (new_path, new_name)


In [32]:


# Checking collision after renaming
path_set = {}

for old_path in rename_table:
    new_path, new_name = rename_table[old_path]
    
    if new_path in existing_files_set:
        if new_path in path_set:
            path_set[new_path].add(new_path)
        else:
            path_set[new_path] = set([new_path])

    if new_path in path_set:
        path_set[new_path].add(old_path)
    else:
        path_set[new_path] = set([old_path])

collision_table = {k: path_set[k] for k in path_set if len(path_set[k]) > 1 }

collision_table


{}

In [16]:
# Not needed now. Checking and fixing collisions while renaming files

## fixing collisions manually

# rename_table['files/D-26-2018-19 revised infosheet.pdf'] = ('files/D-26-2018-19_revised_infosheet_001.pdf', 'D-26-2018-19_revised_infosheet_001.pdf')
# rename_table['files/D-26-2018-19 revised_infosheet.pdf'] = ('files/D-26-2018-19_revised_infosheet_002.pdf', 'D-26-2018-19_revised_infosheet_002.pdf')

# rename_table['files/tenders/2020-12/Tech Spec  Switch  NC  SFP Cable_0.pdf'] = ('files/tenders/2020-12/Tech_Spec__Switch__NC__SFP_Cable_0_001.pdf', 'Tech_Spec__Switch__NC__SFP_Cable_0_001.pdf')
# rename_table['files/tenders/2020-12/Tech Spec, Switch, NC, SFP Cable_0.pdf'] = ('files/tenders/2020-12/Tech_Spec__Switch__NC__SFP_Cable_0_002.pdf', 'Tech_Spec__Switch__NC__SFP_Cable_0_002.pdf')

# rename_table['files/tenders/2020-12/NIT ( 6100000518).pdf'] = ('files/tenders/2020-12/NIT___6100000518_001.pdf', 'NIT___6100000518_001.pdf')
# rename_table['files/tenders/2020-12/NIT_( 6100000518).pdf'] = ('files/tenders/2020-12/NIT___6100000518_002.pdf', 'NIT___6100000518_002.pdf')

# rename_table['files/tenders/2020-02/Technical Specifications 2.pdf'] = ('files/tenders/2020-02/Technical_Specifications_2_001.pdf', 'Technical_Specifications_2_001.pdf')
# rename_table['files/tenders/2020-02/Technical Specifications_2.pdf'] = ('files/tenders/2020-02/Technical_Specifications_2_002.pdf', 'Technical_Specifications_2_002.pdf')

# rename_table['files/jobs/2019-08/Detailed Advt Part time Physician and Visiting Orthopedic Surgeon.pdf'] = ('files/jobs/2019-08/Detailed_Advt_Part_time_Physician_and_Visiting_Orthopedic_Surgeon_001.pdf', 'Detailed_Advt_Part_time_Physician_and_Visiting_Orthopedic_Surgeon_001.pdf')
# rename_table['files/jobs/2019-08/Detailed Advt_Part time Physician and Visiting Orthopedic Surgeon.pdf'] = ('files/jobs/2019-08/Detailed_Advt_Part_time_Physician_and_Visiting_Orthopedic_Surgeon_002.pdf', 'Detailed_Advt_Part_time_Physician_and_Visiting_Orthopedic_Surgeon_002.pdf')


In [33]:

logFile = open("operations.log", 'at')

for old_path in invalid_file_table:
    new_path, new_name = rename_table[old_path]

    ret_mv = os.system(f'git mv "{old_path}" "{new_path}"')

    if ret_mv != 0:
        print(f"Error with renaming file: {old_path}")
        break
    
    # Add to log
    logFile.write(f'{old_path}::renamed to::{new_path}\n')

    encoded_new_path = quote(new_path)

    for csv_file_name, col, row in invalid_file_table[old_path][2]:
        csv_file = csvs[csv_file_name]

        orig_cont = csv_file.iloc[row, col]
        replaced_cont = orig_cont.replace(invalid_file_table[old_path][0], encoded_new_path)

        csv_file.iloc[row, col] = replaced_cont

        logFile.write(f"{csv_file_name}:{row},{col}::\n")
        logFile.write(f"{orig_cont}\n")
        logFile.write(f"{replaced_cont}\n\n")

logFile.close()



Error with renaming file: files/DATA SUBMITTED FOR ENGINEERING CATEGORY.PDF


fatal: destination exists, source=files/DATA SUBMITTED FOR ENGINEERING CATEGORY.PDF, destination=files/DATA_SUBMITTED_FOR_ENGINEERING_CATEGORY.PDF


In [41]:
[x for x in existing_files_set if '(2)' in x]

['files/article/images/pasted image 0 (2).png',
 'files/tenders/2021-07/Technical Specifications(2).pdf',
 'files/news/2018-04/IMG_0886 (2).JPG',
 'files/styles/news_images/public/news/2018-04/IMG_0886 (2).JPG',
 'files/jobs/2017-02/Rect-Admn-II-2016-12(2).pdf',
 'files/styles/large/public/news/2018-03/275A5264 (2).JPG',
 'files/tenders/2014-12/rfp_eAuction (2).pdf',
 'files/tenders/2022-01/Corrigendum I (2).pdf',
 'files/tenders/2022-07/Technical specification (2).pdf',
 'files/styles/full_hd/public/article/images/pasted image 0 (2).png',
 'files/news/2018-03/275A5264 (2).JPG',
 'files/tenders/2014-12/rfp_eAuction (2).odt',
 'files/tenders/2022-06/price Bid_merged (2).pdf',
 'files/tenders/2021-04/Formats (2).pdf',
 'files/tenders/2021-06/Technical Specification for RS Seismogrpah (2).pdf',
 'files/tenders/2020-11/Technical Specifications (2).pdf',
 'files/article/images/WhatsApp Image 2018-08-26 at 07.06.34 (2).jpeg',
 'files/styles/medium/public/article/images/WhatsApp Image 2018-08